In [109]:
# Third Party libraries
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd

from sklearn.linear_model import LinearRegression

In [110]:
# Import Data_cleaning module
from modules.data_clean import DataCleaner

# Load DF
df = pd.read_csv('../../data/raw/Fire_Incidents_Data.csv', low_memory=False)

In [111]:
# Cleanse Dataframe
df = DataCleaner.cleanse_dataframe(df)

In [112]:
df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
3,"75 - Trash, rubbish area (outside)",01 - Normal (no change),1 - No business interruption,0.0,0.0,0.0,0.0,1 - Confined to object of origin,01 - Fire,9 - Undetermined,8 - Not applicable (no system),9 - Undetermined,999 - Undetermined,5.0,Fire - Commercial/Industrial,99 - Undetermined (formerly 98),1 - Extinguished by fire department,6.0,22.0,99 - Undetermined,511 - Department Store,9 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,9 - Type undetermined,"8 - Not applicable: No alarm, no persons present",99 - Undetermined,8 - Not applicable - no sprinkler system present,9 - Undetermined,3 - Fire with smoke showing only - including v...,0.0
6,22 - Sleeping Area or Bedroom (inc. patients r...,01 - Normal (no change),1 - No business interruption,0.0,0.0,2000.0,0.0,2 - Confined to part of room/area of origin,01 - Fire,"8 - Not applicable: No fire alarm system, no p...",8 - Not applicable (no system),8 - Not applicable (bldg not classified by OBC...,"51 - Incandescent Lamp - Light Bulb, Spotlight",19.0,Fire - Residential,16 - Insulation,1 - Extinguished by fire department,6.0,22.0,20 - Design/Construction/Installation/Maintena...,301 - Detached Dwelling,1 - Floor/suite of fire origin: No smoke alarm,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,7 - Not applicable: Occupant(s) first alerted ...,2 - Confined to part of room/area of origin,8 - Not applicable - no sprinkler system present,3 - No sprinkler system,2 - Fire with no evidence from street,0.0
7,55 - Mechanical/Electrical Services Room,01 - Normal (no change),2 - May resume operations within a week,0.0,0.0,100000.0,0.0,2 - Confined to part of room/area of origin,01 - Fire,2 - Some persons (at risk) evacuated as a resu...,1 - Fire alarm system operated,1 - Fire alarm system present,23 - Distribution Equipment (includes panel bo...,24.0,Alarm Highrise Residential,43 - Electrical Wiring Insulation,1 - Extinguished by fire department,24.0,71.0,52 - Electrical Failure,323 - Multi-Unit Dwelling - Over 12 Units,9 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,8 - Not applicable - no smoke alarm or presenc...,2 - Some persons (at risk) self evacuated as a...,"7 - Spread to other floors, confined to building",8 - Not applicable - no sprinkler system present,3 - No sprinkler system,2 - Fire with no evidence from street,0.0
8,28 - Office,01 - Normal (no change),1 - No business interruption,0.0,0.0,5000.0,0.0,1 - Confined to object of origin,01 - Fire,1 - All persons (at risk of injury) evacuated ...,1 - Fire alarm system operated,1 - Fire alarm system present,41 - Other Heating Equipment,10.0,Alarm Commercial/Industrial,"56 - Paper, Cardboard",1 - Extinguished by fire department,8.0,20.0,46 - Used or Placed too close to combustibles,156 - Court Facility,2 - Floor/suite of fire origin: Smoke alarm pr...,98 - Not applicable: Alarm operated OR presenc...,2 - Hardwired (standalone),1 - All persons (at risk of injury) self evacu...,"4 - Spread beyond room of origin, same floor",3 - Did not activate: fire too small to trigge...,1 - Full sprinkler system present,4 - Flames showing from small area (o

### Encoding categorical features

In [113]:
CATEGORICAL_COLS = ['Area_of_Origin', 'Building_Status', 'Business_Impact', 'Extent_Of_Fire', 'Final_Incident_Type', 
                'Fire_Alarm_System_Impact_on_Evacuation', 'Fire_Alarm_System_Operation', 'Fire_Alarm_System_Presence', 
                'Ignition_Source', 'Initial_CAD_Event_Type', 'Material_First_Ignited', 'Method_Of_Fire_Control', 
                'Possible_Cause', 'Property_Use', 'Smoke_Alarm_at_Fire_Origin', 'Smoke_Alarm_at_Fire_Origin_Alarm_Failure', 
                'Smoke_Alarm_at_Fire_Origin_Alarm_Type', 'Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation', 
                'Smoke_Spread', 'Sprinkler_System_Operation', 'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival']

# ['Business_Impact', 'Extent_Of_Fire', 'Sprinkler_System_Presence', 'Status_of_Fire_On_Arrival']
ORDINAL_COLS = CATEGORICAL_COLS[2:4] + CATEGORICAL_COLS[20:22]

# Everything else.
ONEHOT_COLS = list(set(CATEGORICAL_COLS) - set(ORDINAL_COLS))

Encoding all categorical columns as ordinal

This yields bad results.

In [114]:
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
import numpy as np

# Specify ordinal encoder
ordinal_encoder = OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan)

# Apply the encoder to specified columns
df[CATEGORICAL_COLS] = ordinal_encoder.fit_transform(df[CATEGORICAL_COLS])
df

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
3,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,3.0,82.0,5.0,54.0,52.0,0.0,6.0,22.0,22.0,119.0,8.0,9.0,5.0,5.0,9.0,5.0,3.0,2.0,0.0
6,6.0,0.0,0.0,0.0,0.0,2000.0,0.0,3.0,0.0,5.0,2.0,2.0,46.0,19.0,66.0,5.0,0.0,6.0,22.0,6.0,62.0,0.0,9.0,4.0,4.0,1.0,5.0,2.0,1.0,0.0
7,36.0,0.0,1.0,0.0,0.0,100000.0,0.0,3.0,0.0,1.0,0.0,0.0,19.0,24.0,4.0,21.0,0.0,24.0,71.0,16.0,71.0,8.0,9.0,4.0,1.0,6.0,5.0,2.0,1.0,0.0
8,12.0,0.0,0.0,0.0,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,10.0,1.0,32.0,0.0,8.0,20.0,10.0,33.0,2.0,9.0,1.0,0.0,3.0,2.0,0.0,3.0,0.0
11,8.0,0.0,5.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,13.0,66.0,42.0,2.0,6.0,21.0,8.0,65.0,2.0,9.0,0.0,0.0,1.0,5.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29129,5.0,0.0,5.0,1.0,1.0,100000.0,1.0,3.0,0.0,5.0,2.0,2.0,58.0,24.0,46.0,56.0,0.0,20.0,66.0,27.0,62.0,3.0,9.0,0.0,3.0,3.0,5.0,2.0,2.0,0.0
29196,5.0,0.0,5.0,1.0,1.0,100.0,0.0,3.0,0.0,1.0,0.0,0.0,30.0,12.0,25.0,57.0,4.0,8.0,28.0,26.0,72.0,3.0,9.0,3.0,1.0,3.0,5.0,2.0,1.0,0.0
29261,8.0,0.0,5.0,1.0,0.0,500000.0,997.0,9.0,0.0,6.0,2.0,2.0,82.0,14.0,46.0,60.0,0.0,47.0,149.0,28.0,63.0,9.0,9.0,5.0,6.0,7.0,5.0,2.0,2.0,0.0
29285,22.0,0.0,5.0,1.0,0.0,80000.0,0.0,5.0,0.0,5.0,2.0,1.0,82.0,6.0,46.0,58.0,4.0,45.0,134.0,25.0,62.0,3.0,9.0,5.0,2.0,7.0,5.0,2.0,7.0,0.0


Encoding onehot columns - Work in progress/Not working.

In [115]:


# onehot_encoder = OneHotEncoder(handle_unknown="ignore")
# # Apply the imputer to the categorical columns
# df_encoded= onehot_encoder.fit_transform(df[ONEHOT_COLS])

# df_encoded

Simple Imputer on all categorical columns 

In [116]:

# Specify pd.NA as the missing value indicator for SimpleImputer
imputer = SimpleImputer(strategy='most_frequent', missing_values=pd.NA)

# Apply the imputer to the categorical columns
df[CATEGORICAL_COLS] = imputer.fit_transform(df[CATEGORICAL_COLS])
df
        

,Area_of_Origin,Building_Status,Business_Impact,Civilian_Casualties,Count_of_Persons_Rescued,Estimated_Dollar_Loss,Estimated_Number_Of_Persons_Displaced,Extent_Of_Fire,Final_Incident_Type,Fire_Alarm_System_Impact_on_Evacuation,Fire_Alarm_System_Operation,Fire_Alarm_System_Presence,Ignition_Source,Incident_Ward,Initial_CAD_Event_Type,Material_First_Ignited,Method_Of_Fire_Control,Number_of_responding_apparatus,Number_of_responding_personnel,Possible_Cause,Property_Use,Smoke_Alarm_at_Fire_Origin,Smoke_Alarm_at_Fire_Origin_Alarm_Failure,Smoke_Alarm_at_Fire_Origin_Alarm_Type,Smoke_Alarm_Impact_on_Persons_Evacuating_Impact_on_Evacuation,Smoke_Spread,Sprinkler_System_Operation,Sprinkler_System_Presence,Status_of_Fire_On_Arrival,TFS_Firefighter_Casualties
3,56.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,2.0,3.0,82.0,5.0,54.0,52.0,0.0,6.0,22.0,22.0,119.0,8.0,9.0,5.0,5.0,9.0,5.0,3.0,2.0,0.0
6,6.0,0.0,0.0,0.0,0.0,2000.0,0.0,3.0,0.0,5.0,2.0,2.0,46.0,19.0,66.0,5.0,0.0,6.0,22.0,6.0,62.0,0.0,9.0,4.0,4.0,1.0,5.0,2.0,1.0,0.0
7,36.0,0.0,1.0,0.0,0.0,100000.0,0.0,3.0,0.0,1.0,0.0,0.0,19.0,24.0,4.0,21.0,0.0,24.0,71.0,16.0,71.0,8.0,9.0,4.0,1.0,6.0,5.0,2.0,1.0,0.0
8,12.0,0.0,0.0,0.0,0.0,5000.0,0.0,0.0,0.0,0.0,0.0,0.0,37.0,10.0,1.0,32.0,0.0,8.0,20.0,10.0,33.0,2.0,9.0,1.0,0.0,3.0,2.0,0.0,3.0,0.0
11,8.0,0.0,5.0,0.0,0.0,500.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,13.0,66.0,42.0,2.0,6.0,21.0,8.0,65.0,2.0,9.0,0.0,0.0,1.0,5.0,2.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29129,5.0,0.0,5.0,1.0,1.0,100000.0,1.0,3.0,0.0,5.0,2.0,2.0,58.0,24.0,46.0,56.0,0.0,20.0,66.0,27.0,62.0,3.0,9.0,0.0,3.0,3.0,5.0,2.0,2.0,0.0
29196,5.0,0.0,5.0,1.0,1.0,100.0,0.0,3.0,0.0,1.0,0.0,0.0,30.0,12.0,25.0,57.0,4.0,8.0,28.0,26.0,72.0,3.0,9.0,3.0,1.0,3.0,5.0,2.0,1.0,0.0
29261,8.0,0.0,5.0,1.0,0.0,500000.0,997.0,9.0,0.0,6.0,2.0,2.0,82.0,14.0,46.0,60.0,0.0,47.0,149.0,28.0,63.0,9.0,9.0,5.0,6.0,7.0,5.0,2.0,2.0,0.0
29285,22.0,0.0,5.0,1.0,0.0,80000.0,0.0,5.0,0.0,5.0,2.0,1.0,82.0,6.0,46.0,58.0,4.0,45.0,134.0,25.0,62.0,3.0,9.0,5.0,2.0,7.0,5.0,2.0,7.0,0.0


Standard scaler on data. WIP 

In [117]:
from sklearn.preprocessing import StandardScaler

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the DataFrame
#df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
#df

### Train test split

In [118]:
from sklearn.model_selection import train_test_split

# List all columns in the DataFrame
all_columns = df.columns.tolist()

# Use every other column in the df except for the response variable
features = [col for col in all_columns if col != 'Estimated_Dollar_Loss']

# Separate features from response variable
X, y = df[features], df['Estimated_Dollar_Loss']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

### Apply logistic regressor (no pipeline)

In [119]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Instantiate the Logistic Regression model
logreg = LogisticRegression()

# Fit the model on the training data
logreg.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logreg.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.1769499417927823


c:\Users\Zer0v\OneDrive - my.yorku.ca\ITEC4305\Project\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Create mini-pipeline
Applies the following:
- Ordinal encoding
- One hot encoding
- Imputes the data 
- Logistic regressor

In [83]:
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score

# # Create a pipeline with ColumnTransformer, OneHotEncoder, and LogisticRegression
# pipeline = Pipeline([
#     ('preprocessor', ColumnTransformer([
#         ('ordinal', OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=np.nan), ORDINAL_COLS),
#         ('onehot', OneHotEncoder(handle_unknown='ignore'), ONEHOT_COLS),
#         ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA), ONEHOT_COLS + ORDINAL_COLS)
#     ], remainder='passthrough')),
#     ('classifier', LogisticRegression(max_iter=1000))
# ])


### Apply Logistic Regressor (pipeline)

In [ ]:

pipeline.fit(X_train, y_train)

# Make predictions on the test data
y_pred = pipeline.predict(X_test)

# Evaluate the model (e.g., using accuracy score)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)